<a href="https://colab.research.google.com/github/jacobfvanzyl/KML_Extractor/blob/master/Point_in_Polygon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XSIT KML Validator

## How-to:
1. Link Google Drive to Colab
1. Copy path to a folder on GDrive where files to be imported are located
1. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Environment setup

In [0]:
# Only run this cell on every new instance
!pip install pandas
!pip install numpy
!pip install fastkml
!pip install xmltodict
!pip install pprint

debug = True
%pwd

## Package imports

In [0]:
import os
import numpy as np
import pandas as pd
from fastkml import kml
import xmltodict
import json
from shapely.geometry import Point, Polygon
from shapely import speedups

import pprint
pp = pprint.PrettyPrinter(indent=4)

## Paths

In [0]:
batch_path = "drive/My Drive/PointInPolygon/"
batch_name = "SRV SUB Areas - Contiuously edited"
points_name = "TRAPS 2019-2020 Master"
input_path = batch_path + "input/"
output_path = batch_path + "output/"
debug_path = batch_path + 'output/debug/'

## Load KML data

In [6]:
# Polygons file
kml_path = input_path + batch_name + ".kml"
json_path = input_path + batch_name + ".json"

# Points file
points_kml_path = input_path + points_name + ".kml"
points_json_path = input_path + points_name + ".json"

os.makedirs(os.path.dirname(debug_path), exist_ok=True)

data = {}
points_data = {}

# Write XML to JSON
with open(kml_path) as xml_file:
    with open(json_path, 'w') as json_file:
        json.dump(xmltodict.parse(xml_file.read()), json_file)

with open(points_kml_path) as points_xml_file:
    with open(points_json_path, 'w') as points_json_file:
        json.dump(xmltodict.parse(points_xml_file.read()), points_json_file)

# Traverse down into root level
with open(points_json_path) as point_json_data:
    point_contents = point_json_data.read()
    points_data = json.loads(point_contents)
    points_data = points_data["kml"]["Document"] # <-- Never do this iteratively

with open(json_path) as json_data:
    contents = json_data.read()
    data = json.loads(contents)
    data = data["kml"]["Document"]

print("Successfully created JSON file and loaded files: " + data['name'] + ", " + points_data['name'] + " into memory")

points_data = points_data['Folder']['Folder']
data = data['Folder']['Folder']

if debug:
    # print(json.dumps(data, indent=2, sort_keys=True))
    with open(debug_path + 'polygons.json', 'w') as poly_file:
        json.dump(data, poly_file)

    with open(debug_path + 'points.json', 'w') as points_file:
        json.dump(points_data, points_file)

Successfully created JSON file and loaded files: SIT NEW SUB-AREAS.kml, TRAPS 2019-2020 Master.kml into memory


## Data levels

Set the levels in the list below to correspond with the structure of the current KML file

In [0]:
# Levels
data_levels = ['Region', 'Organisation', 'Farm', 'Block', 'Polygon']

point_levels = ['Organisation', 'Zone', 'Coords', 'Altitude']

combined_levels = ['Region', 'Organisation', 'Farm', 'Block', 'Polygon', 'ZONE', 'COORDS']

## Nested data extraction

Traverse and extract nested JSON data and insert into Pandas DataFrame

### Polygons

In [0]:
# Polygons

def extractPolygon(level, block):
    
    if debug:
        print("----- P: " + block['Polygon']['outerBoundaryIs']['LinearRing']['coordinates'])
        
    coord_list = list(block['Polygon']['outerBoundaryIs']['LinearRing']['coordinates'].replace(',0 ', ';').replace(',0', '').split(";"))
    
    formatted_list = []

    for i in coord_list:
        pair = i.split(",")
        lat = float(pair[1])
        lon = float(pair[0])
        flip_pair = [lat, lon]
        formatted_list.append(flip_pair.copy())

    row[data_levels[level]] = formatted_list
    polygon_table.append(row.copy())

    # print(row['Polygon'])


def extractBlock(level, parent): 
    if type(parent['Placemark']) is dict:

        if debug:
            print("---- B: " + parent['Placemark']['name'])

        row[data_levels[level]] = str(parent['Placemark']['name'])

        extractPolygon((level + 1), parent['Placemark'])
                
    elif type(parent['Placemark']) is list:

        for block in parent['Placemark']:

            if debug:
                print("---- B: " + block['name'])

            row[data_levels[level]] = str(block['name'])

            extractPolygon((level + 1), block)

def extractFarm(level, parent):
    if type(org['Folder']) is dict:     # - Single
        
        if debug:
            print("--- F: " + org['Folder']['name'])

        row[data_levels[level]] = org['Folder']['name']

        if 'Placemark' in org['Folder']:
            extractBlock((level + 1), org['Folder']) 
                        
        else:
            if debug:
                print('!--- B: No nested Blocks in Farm')

    elif type(org['Folder']) is list:   # - List

        for farm in org['Folder']:
            
            if debug:
                print("--- F: " + farm['name'])
            
            row[data_levels[level]] = farm['name'] 

            if 'Placemark' in farm:
                extractBlock((level + 1), farm)   

            else:
                print('!--- B: No nested Blocks in Farm')


polygon_table = []

count_regions = 0
count_orgs = 0
count_farms = 0
count_blocks = 0

for region in data:    

    row = {}
    level = 0

    if debug:
        print("- R: " + region['name'])

    row[data_levels[level]] = region['name'] 
    count_regions += 1

    for org in region['Folder']:

        if debug:
            print("-- O: " + org['name'])
        
        row[data_levels[(level + 1)]] = org['name']
        
        if "Folder" in org:     # FARMS
            extractFarm((level + 2), org)

        else:
            print("!-- F: No nested Farms in Org")

# print(polygon_table)

In [11]:
# Print Polygons DataFrame
poly_df = pd.DataFrame(polygon_table)
poly_df.to_excel(output_path + batch_name + '.xlsx', index=False)
poly_df

,Region,Organisation,Farm,Block,Polygon
0,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Knightsmead,1,"[[-33.41098338567136, 25.43725693096086], [-33..."
1,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Knightsmead,3,"[[-33.40790497369915, 25.43802584269752], [-33..."
2,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,2,"[[-33.40219220166004, 25.38933659375389], [-33..."
3,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,4,"[[-33.40234544132873, 25.38852443959581], [-33..."
4,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,5,"[[-33.402532895722, 25.38743564794471], [-33.4..."
...,...,...,...,...,...
2919,BARKLEY BRIDGE,Wreford Hudson,Rawdon 2,21,"[[-33.56286343006675, 25.69428371176782], [-33..."
2920,BARKLEY BRIDGE,Wreford Hudson,Boplaas 1,30,"[[-33.56259675787091, 25.69981656874453], [-33..."
2921,BARKLEY BRIDGE,Wreford Hudson,Boplaas 1,32,"[[-33.56414799884166, 25.69945207277419], [-33..."
2922,BARKLEY BRIDGE,Wreford Hudson,Boplaas 2,26,"[[-33.56354771695999, 25.69965138493305], [-33..."


### Points

In [0]:
# Points
points_table = []

for org in points_data:

    row = {}

    if debug:
        print("- O: " + org['name'])
    
    row[point_levels[0]] = org['name']
    
    if 'Placemark' in org:
            if type(org['Placemark']) is dict:
                if debug:
                    print("-- Z: " + org['Placemark']['name'])
                row[point_levels[1]] = org['Placemark']['name']
                if debug:
                    print("--- C: " + org['Placemark']['Point']['coordinates'])
                pair = list(org['Placemark']['Point']['coordinates'].split(","))
                lat = float(pair[1])
                lon = float(pair[0])
                alt = float(pair[2])
                flipped_pair = [lat, lon]
                
                row[point_levels[2]] = flipped_pair
                row[point_levels[3]] = alt

                points_table.append(row.copy())
                print(row)
                
            elif type(org['Placemark']) is list:

                for zone in org['Placemark']:
                    if debug:
                        print("-- Z: " + zone['name'])
                    row[point_levels[1]] = zone['name']
                    if debug:
                        print("--- C: " + zone['Point']['coordinates'])
                    pair = list(zone['Point']['coordinates'].split(","))
                    lat = float(pair[1])
                    lon = float(pair[0])
                    alt = float(pair[2])
                    flipped_pair = [lat, lon]

                    row[point_levels[2]] = flipped_pair
                    row[point_levels[3]] = alt

                    points_table.append(row.copy())
                    print(row)
                    
    else:
        print('!--- B: No nested Zones in Org')
        
print(points_table)

In [13]:
# Print Points DataFrame
point_df = pd.DataFrame(points_table)
point_df.to_excel(output_path + points_name + '.xlsx', index=False)
point_df

,Organisation,Zone,Coords,Altitude
0,Adrian Walton,AWL - LDV 1 - 8 (BD/FF-F),"[-33.446451, 25.555643]",64.96
1,Adrian Walton,AWL - LDV 2 - 5 (FF-M),"[-33.44787051268876, 25.55910193195675]",66.08
2,Adrian Walton,AWL - LDN 1 - 19 (FF-M),"[-33.446825, 25.560775]",71.39
3,Adrian Walton,AWL - LDN 2 - 21 (BD/FF-F),"[-33.446164, 25.562659]",68.94
4,Adrian Walton,AWL - LDN 3 - 27 (FF-M),"[-33.446816, 25.566374]",70.73
...,...,...,...,...
2191,Wreford Hudson,WHU - SAS D5 - 12,"[-33.40332549016711, 25.45244604297021]",0.00
2192,Wreford Hudson,WHU - VOL 1 - 23 (BD/FF),"[-33.439341, 25.482755]",82.86
2193,Wreford Hudson,WHU - VOL 2 - 30 (BD/FF),"[-33.447583, 25.492902]",78.90
2194,Wreford Hudson,WHU - VOL D1 - 14,"[-33.44098962657348, 25.48247708571547]",0.00


## Point-in-Polygon match

In [180]:
%%time

matched_table = []

for zone in point_df.index:
    
    row = {}

    point = Point(point_df[point_levels[2]][zone][0], point_df[point_levels[2]][zone][1])
    
    match = False
    count = 0

    for block in poly_df.index:
        poly = Polygon(poly_df[poly_df[data_levels[4]][block])
        
        if point.within(poly):
            
            match = True
            count += 1

            row[combined_levels[0]] = poly_df[data_levels[0]][block]
            row[combined_levels[1]] = poly_df[data_levels[1]][block]
            row[combined_levels[2]] = poly_df[data_levels[2]][block]
            row[combined_levels[3]] = poly_df[data_levels[3]][block]
            row[combined_levels[4]] = poly_df[data_levels[4]][block]
            row[combined_levels[5]] = point_df[point_levels[1]][zone]
            row[combined_levels[6]] = point_df[point_levels[2]][zone]

            matched_table.append(row.copy())

    if match == True:
        print(str(row) + '  | Count: ' + str(count))

    elif match == False:
        print('Zone: ' + point_df['Organisation'][zone] + ', ' + point_df['Zone'][zone] + ' has NO MATCH.')

{'Region': 'DUNBRODY', 'Organisation': 'Bernol Farming (Pty) Ltd', 'Farm': 'Lupus Den Vlei', 'Block': '8', 'Polygon': [[-33.44555093433758, 25.55519753152435], [-33.44673795066535, 25.55464974576304], [-33.44707108497601, 25.55575240058439], [-33.44588824724875, 25.55625680674299], [-33.44555093433758, 25.55519753152435]], 'ZONE': 'AWL -  LDV 1 - 8 (BD/FF-F)', 'COORDS': [-33.446451, 25.555643]}  | Count: 1
{'Region': 'DUNBRODY', 'Organisation': 'Bernol Farming (Pty) Ltd', 'Farm': 'Lupus Den Vlei', 'Block': '5', 'Polygon': [[-33.44769786695151, 25.55786312166964], [-33.44811939815688, 25.55921505003152], [-33.44690062640697, 25.55976495232736], [-33.44647458754005, 25.55841700408907], [-33.44769786695151, 25.55786312166964]], 'ZONE': 'AWL - LDV 2 - 5 (FF-M)', 'COORDS': [-33.44787051268876, 25.55910193195675]}  | Count: 1
{'Region': 'DUNBRODY', 'Organisation': 'Bernol Farming (Pty) Ltd', 'Farm': 'AD Lupus Den', 'Block': '19', 'Polygon': [[-33.44674333316566, 25.55951390641798], [-33.4471

## Output

In [125]:
matched_df = pd.DataFrame(matched_table)
matched_df.to_excel(output_path + 'matched.xlsx', index=False)
matched_df

,Region,Organisation,Farm,Block,Polygon,ZONE,COORDS
0,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Knightsmead,1,"[[-33.41098338567136, 25.43725693096086], [-33...",APG - KNI D2 - 1,"[-33.41025047588008, 25.43782908571556]"
1,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Knightsmead,3,"[[-33.40790497369915, 25.43802584269752], [-33...",APG - KNI D1 - 3,"[-33.40909322765108, 25.43840238308366]"
2,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,2,"[[-33.40219220166004, 25.38933659375389], [-33...",APG - SWA 2 - 2 (FF-M),"[-33.402054, 25.389096]"
3,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,4,"[[-33.40234544132873, 25.38852443959581], [-33...",APG - SWA D1 - 4,"[-33.40231628552035, 25.3882433187815]"
4,KIRKWOOD-WES,Swartkop Boerdery (Pty) Ltd,Swartkop,5,"[[-33.402532895722, 25.38743564794471], [-33.4...",APG - SWA 1 - 5 (FF-F),"[-33.402363, 25.387149]"
...,...,...,...,...,...,...,...
2700,BARKLEY BRIDGE,Wreford Hudson,Rawdon 2,21,"[[-33.56286343006675, 25.69428371176782], [-33...",WHU - RAW D1 - 21 (BD),"[-33.56306115839159, 25.69681066928683]"
2701,BARKLEY BRIDGE,Wreford Hudson,Boplaas 1,30,"[[-33.56259675787091, 25.69981656874453], [-33...",WHU - BOP D1 - 30,"[-33.56274723564458, 25.6973241294694]"
2702,BARKLEY BRIDGE,Wreford Hudson,Boplaas 1,32,"[[-33.56414799884166, 25.69945207277419], [-33...",WHU - BOP 1 - 32 (BD/FF),"[-33.56408336076098, 25.69707834234337]"
2703,BARKLEY BRIDGE,Wreford Hudson,Boplaas 2,26,"[[-33.56354771695999, 25.69965138493305], [-33...",WHU - BOP D2 - 26 (BD),"[-33.56403010070868, 25.69987741845922]"
